# Neural Collaborative Filtering (NCF) and Neural Matrix Factorization (NeuMF) Recommendation Systems

## Overview

Collaborative Filtering is a popular technique in recommendation systems that leverages user-item interactions to make personalized recommendations. Neural Collaborative Filtering (NCF) and Neural Matrix Factorization (NeuMF) are advanced models that incorporate neural networks to enhance collaborative filtering.

## Neural Collaborative Filtering (NCF)

NCF is a collaborative filtering model that integrates neural networks into the traditional collaborative filtering framework. It is designed to capture complex patterns and non-linear relationships in user-item interactions. NCF utilizes neural networks to model both user and item embeddings, combining the strengths of collaborative filtering and deep learning.

[Link to NCF and NeuMF Paper](papers/NeuMF.pdf)

### Architecture

NCF typically consists of the following components:
- **Embedding Layers:** Embeds users and items into low-dimensional latent vectors.
- **Neural Network Layers:** Processes the embedded vectors to capture non-linear patterns.
- **Output Layer:** Generates a prediction score indicating the likelihood of user-item interactions.

### Key Advantages
- **Flexibility:** NCF can capture intricate user-item relationships, including implicit feedback.
- **Scalability:** The neural network structure allows for scalability with large datasets.


<div style="text-align:center">
    <img src="assets/NCF.png" alt="NCF" width="70%">
</div>

In [1]:
import torch.nn as nn
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class DataPreprocessor:
    def __init__(self, file_path='datasets/ml-1m/movies.dat', test_size=0.2):
        # Define the column names
        columns = ['userId', 'movieId', 'rating', 'timestamp']
        self.test_size = test_size
        # Read the .dat file using pandas
        self.ratings = pd.read_csv(file_path, sep='::', header=None, names=columns, engine='python', encoding='latin-1')
        self.ratings['timestamp'] = pd.to_datetime(self.ratings['timestamp'], unit='s')
        self.user_mapping = {user_id: idx for idx, user_id in enumerate(self.ratings['userId'].unique())}
        self.reverse_user_mapping = {idx: user_id for idx, user_id in enumerate(self.ratings['userId'].unique())}
        self.movie_mapping = {movie_id: idx for idx, movie_id in enumerate(self.ratings['movieId'].unique())}
        self.reverse_movie_mapping = {idx: movie_id for idx, movie_id in enumerate(self.ratings['movieId'].unique())}
        self.map_user_movie_ids()

    def train_test_split(self):
        # Sort the DataFrame by the 'timestamp' column
        df_sorted = self.ratings.sort_values(by='timestamp')

        train_ratings, test_ratings = [], []

        for _, user_data in df_sorted.groupby('userId'):
            n_samples = len(user_data)
            num_test_samples = int(self.test_size * n_samples)
            if num_test_samples:
                train_ratings.append(user_data.iloc[:-num_test_samples])
                test_ratings.append(user_data.iloc[-num_test_samples:])
            else:
                train_ratings.append(user_data)

        train_ratings = pd.concat(train_ratings)
        test_ratings = pd.concat(test_ratings)
        return train_ratings, test_ratings, self.ratings

    def map_user_movie_ids(self):
        # Map userId and movieId to start from 0 and be incremental
        user_mapping = {user_id: idx for idx, user_id in enumerate(self.ratings['userId'].unique())}
        movie_mapping = {movie_id: idx for idx, movie_id in enumerate(self.ratings['movieId'].unique())}

        self.ratings['userId'] = self.ratings['userId'].map(user_mapping)
        self.ratings['movieId'] = self.ratings['movieId'].map(movie_mapping)


In [3]:
# Example usage:
# Replace 'your_dataset.dat' with the actual file path
data_preprocessor = DataPreprocessor('datasets/ml-1m/ratings.dat')
train_ratings, test_ratings, ratings = data_preprocessor.train_test_split()


In [4]:
# Display the preprocessed DataFrame
print(train_ratings.shape, test_ratings.shape)
print(len(data_preprocessor.ratings['movieId'].unique()), max(data_preprocessor.ratings['movieId'].unique()))
print(len(data_preprocessor.ratings['userId'].unique()), max(data_preprocessor.ratings['userId'].unique()))
print(data_preprocessor.ratings.head())

(802553, 4) (197656, 4)
3706 3705
6040 6039
   userId  movieId  rating           timestamp
0       0        0       5 2000-12-31 22:12:40
1       0        1       3 2000-12-31 22:35:09
2       0        2       3 2000-12-31 22:32:48
3       0        3       4 2000-12-31 22:04:35
4       0        4       5 2001-01-06 23:38:11


In [5]:
class PosDataset(Dataset):
    def __init__(self, ratings):
        self.ratings = ratings[['userId','movieId']].values

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        # Placeholder, modify according to your needs
        user_id, movie_id = self.ratings[idx]
        return {'user_id': user_id, 'item_id': movie_id, 'target': 1}
    
class NegativeSampler(Dataset):
    def __init__(self, ratings, negative_ratio=5, strategy='uniform'):
        self.ratings = ratings
        self.strategy = strategy
        self.negative_ratio = negative_ratio
        self.user_ids = self.ratings['userId'].unique()
        self.item_ids = self.ratings['movieId'].unique()
        self.user_item_set = set(zip(ratings['userId'], ratings['movieId']))
        if self.strategy == 'uniform':
            self.user_freq = None
            self.item_freq = None
        else:
            self.user_freq = self._get_user_freq()
            self.item_freq = self._get_item_freq()

    def __len__(self):
        return self.negative_ratio*len(self.ratings)

    def __getitem__(self, idx):
        # Placeholder, modify according to your needs
        neg_user_id, neg_movie_id = self.negative_sample()
        return {'user_id': neg_user_id, 'item_id': neg_movie_id, 'target': 0}

    def negative_sample(self):
        # Sample negative items
        while True:
            if self.strategy == 'uniform':
                user_id = np.random.choice(self.user_ids)
                neg_item = np.random.choice(self.item_ids)
            else:
                user_id = np.random.choice(self.user_ids, p=self.user_freq)
                neg_item = np.random.choice(self.item_ids, p=self.item_freq)
            if (user_id, neg_item) not in self.user_item_set:
                return user_id, neg_item
    
    def _get_user_freq(self):
        user_counts = self.ratings.groupby('userId').count()['movieId']
        user_probs = user_counts/user_counts.sum()
        user_probs = user_probs.to_dict()
        user_probs_list = [user_probs[id] for id in self.user_ids]
        return user_probs_list
    
    def _get_item_freq(self):
        item_counts = self.ratings.groupby('movieId').count()['userId']
        item_probs = item_counts/item_counts.sum()
        item_probs = item_probs.to_dict()
        item_probs_list = [item_probs[id] for id in self.item_ids]
        return item_probs_list

In [84]:
class TestDataset(Dataset):
    def __init__(self, interactions_df, test_df):
        self.interactions_df = interactions_df
        self.test_df = test_df
        self.users = interactions_df['userId'].unique()

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        user_id = self.users[idx]
        
        # Get all interactions of the user
        user_interactions = self.interactions_df[self.interactions_df['userId'] == user_id]
        
        # Extract the latest interactions as the test set
        test_interaction = self.test_df[self.test_df['userId'] == user_id]
        
        # Sample 100 items that are not interacted by the user
        all_items = self.interactions_df['movieId'].unique()
        interacted_items = user_interactions['movieId'].unique()
        non_interacted_items = np.setdiff1d(all_items, interacted_items)
        sampled_items = np.random.choice(non_interacted_items, size=100, replace=False)
        
        # Create positive and negative samples for evaluation
 
        #pos_array = np.stack([test_interaction['userId'].values, test_interaction['movieId'].values], axis=1)
        #positive_sample = torch.tensor([pos_array], dtype=torch.long)
        pos_user_ids = test_interaction['userId'].values.reshape(-1, 1)
        pos_item_ids = test_interaction['movieId'].values.reshape(-1, 1)

        #negative_samples = torch.tensor([[user_id, item] for item in sampled_items], dtype=torch.long)
        neg_user_ids = np.array([user_id]*len(sampled_items)).reshape(-1, 1)
        neg_item_ids = sampled_items.reshape(-1, 1)
        sample_user_ids = np.vstack([pos_user_ids, neg_user_ids])
        sample_movie_ids = np.vstack([pos_item_ids, neg_item_ids])
        targets = np.vstack([np.ones_like(pos_user_ids), np.zeros_like(neg_user_ids)])
        return {
            'sample_userId': torch.tensor(sample_user_ids), 
            'sample_movieId': torch.tensor(sample_movie_ids), 
            'pos_movieId': torch.tensor(pos_item_ids),
            'targets': torch.tensor(targets)
            }


## Neural Matrix Factorization (NeuMF)

NeuMF is a hybrid recommendation model that combines the strengths of matrix factorization and neural networks. It is designed to improve the recommendation accuracy by leveraging both collaborative filtering and content-based features.

[Link to NCF and NeuMF Paper](papers/NeuMF.pdf)

### Architecture

The architecture of NeuMF includes:
- **Matrix Factorization Component:** Learns user and item embeddings through traditional matrix factorization.
- **Neural Network Component:** Learns additional non-linear patterns using neural networks.
- **Final Prediction:** Combines predictions from both components to produce the final recommendation.

### Key Advantages
- **Hybrid Approach:** NeuMF combines collaborative filtering and neural networks, leveraging the benefits of both.
- **Improved Accuracy:** The model captures both explicit and implicit feedback, enhancing recommendation accuracy.

## Use Cases

NCF and NeuMF find applications in various domains, including:
- **E-commerce:** Personalized product recommendations.
- **Streaming Services:** Content recommendations for users.
- **Social Networks:** Friend or content suggestions.


<div style="text-align:center">
    <img src="assets/NeuMF.png" alt="NCF" width="70%">
</div>

In [7]:
class NCF(nn.Module):
    def __init__(self, user_num, item_num, hidden_dims=[32, 16, 8]):
        super(NCF, self).__init__()
        self.user_emb = nn.Embedding(num_embeddings=user_num, embedding_dim=hidden_dims[0])
        self.item_emb = nn.Embedding(num_embeddings=item_num, embedding_dim=hidden_dims[0])
        
        layers = []
        for i in range(len(hidden_dims)-1):
            layers.append(nn.Linear(in_features=hidden_dims[i], out_features=hidden_dims[i+1]))
            layers.append(nn.Tanh())
        layers.extend([nn.Linear(in_features=hidden_dims[-1], out_features=1), nn.Sigmoid()])
        self.mlp = nn.Sequential(*layers)
        
    def forward(self, user_ids, item_ids):
        user_emb = self.user_emb(user_ids)
        item_emb = self.item_emb(item_ids)
        emb = user_emb*item_emb
        out = self.mlp(emb)
        return out

In [8]:
m = NCF(10, 10)

x1 = torch.randint(low=0, high=10, size=(3, ))
x2 = torch.randint(low=0, high=10, size=(3, ))
t1 = torch.randint(low=0, high=1, size=(3, ))
print(f'x1 shape: {x1.shape}')
m(x1, x2)

x1 shape: torch.Size([3])


tensor([[0.4542],
        [0.4361],
        [0.4390]], grad_fn=<SigmoidBackward0>)

In [9]:
n_items = len(ratings['movieId'].unique())
n_users = len(ratings['userId'].unique())
print(f'Max userId: {n_users} Max movieId: {n_items}')

Max userId: 6040 Max movieId: 3706


In [10]:
model1 = NCF(n_users, n_items)
model1

NCF(
  (user_emb): Embedding(6040, 32)
  (item_emb): Embedding(3706, 32)
  (mlp): Sequential(
    (0): Linear(in_features=32, out_features=16, bias=True)
    (1): Tanh()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): Tanh()
    (4): Linear(in_features=8, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [ ]:
import logging
from datetime import datetime
# Configure logging to write to a file
logging.basicConfig(filename=f'{datetime.now().isoformat()}_log.txt', level=logging.INFO)

In [172]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm


def train(model, epochs, pos_train_dataloader, neg_train_dataloader, test_dataset, lr=0.001):

    # Define Binary Cross Entropy (BCE) loss as the criterion
    criterion = nn.BCELoss()

    # Define the optimizer (e.g., Adam optimizer)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0

        for iter, (pos_batch, ns_batch) in enumerate(tqdm(zip(pos_train_dataloader, neg_train_dataloader), total=len(pos_train_dataloader), desc=f'Epoch {epoch + 1}/{epochs}')):
            user_ids = torch.cat([pos_batch['user_id'], ns_batch['user_id']]).to(device)
            movie_ids = torch.cat([pos_batch['item_id'], ns_batch['item_id']]).to(device)
            targets = torch.cat([pos_batch['target'], ns_batch['target']]).float().to(device)

            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(user_ids, movie_ids)
            outputs = torch.squeeze(outputs, dim=1)
            loss = criterion(outputs, targets)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss / len(pos_train_dataloader)

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {average_loss:.4f}")

        # Evaluate on test set after each epoch
        evaluate(model, test_dataset, k=10)  # You need to define the evaluate function

    print("Training completed.")

def evaluate(model, test_dataset, k):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    hr_sum = 0.0
    ndcg_sum = 0.0
    test_loss = 0.0

    with torch.no_grad():
        for i, data in enumerate(test_dataset):
            user_ids = data['sample_userId'].to(device).squeeze(1)
            movie_ids = data['sample_movieId'].to(device).squeeze(1)
            targets = data['targets'].to(device).squeeze(1)

            outputs = model(user_ids, movie_ids)
            outputs = torch.squeeze(outputs, dim=1)
            # Assuming your model outputs probabilities and you want to use BCE loss
            loss = nn.BCELoss()(outputs, targets.float())

            # Calculate HR@k and NDCG@k
            hr, ndcg = calculate_metrics(outputs, targets, k)

            hr_sum += hr
            ndcg_sum += ndcg
            test_loss += loss.detach().cpu().item()

    average_hr = hr_sum / len(test_dataset)
    average_ndcg = ndcg_sum / len(test_dataset)
    average_test_loss = test_loss/len(test_dataset)
    print('\nEvaluation:')
    print(f"Eval Loss: {average_test_loss:.4f} HR@{k}: {average_hr:.4f}, NDCG@{k}: {average_ndcg:.4f}\n")

def calculate_metrics(outputs, targets, k):
    outputs = outputs.squeeze()
    targets = targets.squeeze()
    
    # Get indices where target is 1
    positive_indices = torch.nonzero(targets).squeeze(dim=1)

    k = min(len(positive_indices), k)
    # Get top-k predicted indices for each user
    _, indices = torch.topk(outputs, k, dim=0)
    
    # Create a binary matrix indicating top-k predictions
    top_k_binary = torch.zeros(k).to(targets.device)
    for i in range(k):
        if indices[i] in positive_indices:
            top_k_binary[i] = 1
    
    
    # Calculate HR@k
    hr_at_k = top_k_binary.sum().item()/k
    
    # Calculate DCG and IDCG for NDCG@k
    dcg = (top_k_binary / torch.log2(torch.arange(2, k + 2).float().to(targets.device))).sum().item()
    idcg = (torch.sort(top_k_binary.clone(), dim=0, descending=True)[0]/ torch.log2(torch.arange(2, k + 2).float().to(targets.device))).sum().item()
    
    # Calculate NDCG@k
    if idcg:
        ndcg_at_k = dcg / idcg
    else:
        ndcg_at_k = 0

    return hr_at_k, ndcg_at_k
        

In [168]:
n_items = len(ratings['movieId'].unique())
n_users = len(ratings['userId'].unique())
print(f'Max userId: {n_users} Max movieId: {n_items}')

Max userId: 6040 Max movieId: 3706


In [169]:
modelNCF = NCF(n_users, n_items)
modelNCF

NCF(
  (user_emb): Embedding(6040, 32)
  (item_emb): Embedding(3706, 32)
  (mlp): Sequential(
    (0): Linear(in_features=32, out_features=16, bias=True)
    (1): Tanh()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): Tanh()
    (4): Linear(in_features=8, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [170]:
negative_ratio = 5
batch_size = 4096
pos_train_dataset  = PosDataset(ratings)
neg_train_dataset = NegativeSampler(ratings, negative_ratio=negative_ratio, strategy='uniform')
test_dateset = TestDataset(ratings, test_ratings)

pos_train_dataloader = DataLoader(pos_train_dataset, batch_size=batch_size)
neg_train_dataloader = DataLoader(neg_train_dataset, batch_size=negative_ratio*batch_size)


In [171]:
train(modelNCF, 5, pos_train_dataloader, neg_train_dataloader, test_dateset, lr=0.001)

Epoch 1/5:   0%|          | 0/245 [00:00<?, ?it/s]

Epoch 1/5: 100%|██████████| 245/245 [01:42<00:00,  2.40it/s]


Epoch [1/5], Loss: 0.5523

Evaluation:
Eval Loss: 0.5101 HR@10: 0.2254, NDCG@10: 0.4742


Epoch 2/5: 100%|██████████| 245/245 [01:41<00:00,  2.41it/s]


Epoch [2/5], Loss: 0.4498

Evaluation:
Eval Loss: 0.5082 HR@10: 0.2866, NDCG@10: 0.5615


...

Epoch [3/5], Loss: 0.4406

Evaluation:
Eval Loss: 0.4864 HR@10: 0.4033, NDCG@10: 0.6659


Epoch 4/5: 100%|██████████| 245/245 [01:41<00:00,  2.41it/s]


Epoch [4/5], Loss: 0.4063

Evaluation:
Eval Loss: 0.4502 HR@10: 0.4463, NDCG@10: 0.6995


Epoch 5/5: 100%|██████████| 245/245 [01:40<00:00,  2.44it/s]


Epoch [5/5], Loss: 0.3766

Evaluation:
Eval Loss: 0.4283 HR@10: 0.4669, NDCG@10: 0.7163
Training completed.


<div style="text-align:center">
    <img src="assets/NeuMF.png" alt="NCF" width="50%">
</div>

In [164]:
import torch
import torch.nn as nn


class NeuMF(nn.Module):
    def __init__(self, user_num, item_num, hidden_dims=[32, 16, 8], log_file='neumf_log.txt'):
        super(NeuMF, self).__init__()

        self.hidden_dims = hidden_dims
        self.user_emb = nn.Embedding(num_embeddings=user_num, embedding_dim=2 * hidden_dims[0])
        self.item_emb = nn.Embedding(num_embeddings=item_num, embedding_dim=2 * hidden_dims[0])

        layers = [nn.Linear(in_features=2 * hidden_dims[0], out_features=hidden_dims[1])]
        for i in range(1, len(hidden_dims) - 1):
            layers.extend([nn.Tanh(), nn.Linear(in_features=hidden_dims[i], out_features=hidden_dims[i + 1])])
        layers.append(nn.Tanh())

        self.mlp = nn.Sequential(*layers)
        self.neumf = nn.Sequential(
            nn.Linear(in_features=hidden_dims[0] + hidden_dims[-1], out_features=1),
            nn.Sigmoid()
        )

    def forward(self, user_ids, item_ids):
        
        user_emb = self.user_emb(user_ids)
        user_emb_mf, user_emb_mlp = user_emb[:, :self.hidden_dims[0]], user_emb[:, self.hidden_dims[0]:]

        item_emb = self.item_emb(item_ids)
        item_emb_mf, item_emb_mlp = item_emb[:, :self.hidden_dims[0]], item_emb[:, self.hidden_dims[0]:]

        emb_mf = user_emb_mf * item_emb_mf

        emb_mlp = torch.cat([user_emb_mlp, item_emb_mlp], dim=1)
        out_mlp = self.mlp(emb_mlp)

        out = torch.cat([emb_mf, out_mlp], dim=1)
        out = self.neumf(out)
        return out


In [173]:
modelNeuMF = NeuMF(n_users, n_items)
modelNeuMF

NeuMF(
  (user_emb): Embedding(6040, 64)
  (item_emb): Embedding(3706, 64)
  (mlp): Sequential(
    (0): Linear(in_features=64, out_features=16, bias=True)
    (1): Tanh()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): Tanh()
  )
  (neumf): Sequential(
    (0): Linear(in_features=40, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [174]:
train(modelNeuMF, 5, pos_train_dataloader, neg_train_dataloader, test_dateset, lr=0.001)

Epoch 1/5:   0%|          | 0/245 [00:00<?, ?it/s]

Epoch 1/5: 100%|██████████| 245/245 [01:41<00:00,  2.42it/s]


Epoch [1/5], Loss: 0.5342

Evaluation:
Eval Loss: 0.4583 HR@10: 0.5178, NDCG@10: 0.7548



Epoch 2/5: 100%|██████████| 245/245 [01:42<00:00,  2.40it/s]


Epoch [2/5], Loss: 0.3737

Evaluation:
Eval Loss: 0.4259 HR@10: 0.5500, NDCG@10: 0.7706



...

Epoch [3/5], Loss: 0.3512

Evaluation:
Eval Loss: 0.4098 HR@10: 0.5505, NDCG@10: 0.7706



Epoch 4/5: 100%|██████████| 245/245 [01:42<00:00,  2.40it/s]


Epoch [4/5], Loss: 0.3380

Evaluation:
Eval Loss: 0.3956 HR@10: 0.5515, NDCG@10: 0.7709



Epoch 5/5: 100%|██████████| 245/245 [01:40<00:00,  2.43it/s]


Epoch [5/5], Loss: 0.3262

Evaluation:
Eval Loss: 0.3839 HR@10: 0.5503, NDCG@10: 0.7702

Training completed.
